In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from xverse.transformer import WOE
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
df = pd.read_csv('credit_scoring_data.csv')


In [ ]:
# Create Aggregate Features
df['Total_Transaction_Amount'] = df.groupby('CustomerID')['TransactionAmount'].transform('sum')
df['Avg_Transaction_Amount'] = df.groupby('CustomerID')['TransactionAmount'].transform('mean')
df['Transaction_Count'] = df.groupby('CustomerID')['TransactionAmount'].transform('count')
df['Std_Transaction_Amount'] = df.groupby('CustomerID')['TransactionAmount'].transform('std').fillna(0)
